In [43]:
import numpy as np
import matplotlib.pyplot as plt
from CPMG.decom_utils import *
from scipy.signal import find_peaks

import os, sys, glob
import tensorflow as tf
import keras
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, CuDNNLSTM, Reshape
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import multi_gpu_model
from keras.models import load_model, Model
from keras import backend as K
from keras.models import load_model

In [44]:
MAGNETIC_FIELD = 403            # Unit: Gauss
GYRO_MAGNETIC_RATIO = 1.07*1000 # Unit: Herts
wl = MAGNETIC_FIELD*GYRO_MAGNETIC_RATIO*2*3.14

t = np.load('../../../../TimTaminiau/20190610_time_no_duplicated.npy')[:13500]*1e-6

In [45]:
def exact(AB_list):
    M_list_temp = 1
    for A, B in AB_list:
        mz = (A + wl) / wl
        mx = B / wl
        w_tilda = pow(pow(A+wl, 2) + B*B, 1/2)
        alpha = w_tilda * t
        beta = wl * t

        phi = np.arccos(np.cos(alpha) * np.cos(beta) - mz * np.sin(alpha) * np.sin(beta))
        K1 = (1 - np.cos(alpha)) * (1 - np.cos(beta))
        K2 = 1 + np.cos(phi)
        K = pow(mx,2) * (K1 / K2)
        M_list_temp *= (1 - K * pow(np.sin(32 * phi/2), 2))
    return (M_list_temp + 1) / 2

def N(A, B):
    M_list_temp = 1
    w_tilda = pow(pow(A+wl, 2) + B*B, 1/2)
    mz = (A + wl) / w_tilda
    mx = B / w_tilda

    alpha = w_tilda * t
    beta = wl * t
    phi = np.arccos(np.cos(alpha) * np.cos(beta) - mz * np.sin(alpha) * np.sin(beta))
    K1 = mx * mx * (1 - np.cos(alpha)) * (1 - np.cos(beta)) / 2
    K2 = 32 * np.cos(8 * phi) * np.cos(4 * phi) * np.cos(2 * phi) * np.cos(phi) * np.sin(phi / 2)
    return K1 * K2**2

In [133]:
def make_heatmap(p):
    A_list = np.linspace(-60000, 60000, 80)
    B_list = np.linspace(2000, 70000, 17)
    indexer = 0
    heatmap = np.zeros((80, 17))
    tp = np.load('/data1/keras_model/filter_indices.npy').item()
    for i, A in enumerate(A_list):
        print(i)
        for j, B in enumerate(B_list):
            dir_name = '/data1/keras_model/' + str(indexer)
            for file in os.listdir(dir_name):
                if file.endswith(".h5"):
                    model = load_model(dir_name + '/' + file)
                    heatmap[i][j] = model.predict(p[tp[indexer]].transpose()[0])[0]
                indexer += 1
    return heatmap

In [134]:
exp_data_tmp = np.load('../../../../TimTaminiau/20190610_exp_data_24200.npy')

In [135]:
hmap = make_heatmap(exp_data_tmp)

0


ValueError: Error when checking input: expected input_1 to have 3 dimensions, but got array with shape (963, 1)

In [120]:
model.predict(exp_data_tmp[tp[2]].transpose())

ValueError: Error when checking input: expected input_2 to have shape (964,) but got array with shape (958,)

In [121]:
tp = np.load('/data1/keras_model/filter_indices.npy').item()

In [131]:
exp_data_tmp[tp[1]].transpose()[0]

array([0.9396762 , 0.96628964, 0.91749834, 0.88201375, 0.85983588,
       0.88201375, 0.89532047, 0.95298292, 0.96628964, 0.94854735,
       0.97918665, 0.94411178, 0.98846751, 0.91306276, 0.94411178,
       0.9396762 , 1.00177423, 0.95298292, 0.97072522, 0.94355956,
       0.96185407, 0.92636948, 0.93524063, 0.9301994 , 0.96185407,
       0.91749834, 0.93465278, 0.97959636, 0.98846751, 0.98846751,
       0.99733866, 0.97516079, 1.00177423, 1.00177423, 0.96582649,
       0.9574185 , 1.00177423, 0.94411178, 0.94411178, 0.90862719,
       0.96185407, 0.91749834, 0.93524063, 0.88201375, 0.91749834,
       0.91306276, 0.90862719, 0.89532047, 0.89532047, 0.89532047,
       0.89532047, 0.86785198, 0.90862719, 0.91749834, 0.87757818,
       0.89975604, 0.86870703, 0.86339859, 0.88644932, 0.85894521,
       0.84209359, 0.78443114, 0.81991572, 0.78886671, 0.90419162,
       0.93524063, 0.94411178, 0.92636948, 0.82878687, 0.60064877,
       0.44732757, 0.33643823, 0.11909514, 0.12353072, 0.18562